In [1]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
import pandas as pd
import requests

In [ ]:
# Coloca tus credenciales aquí
CLIENT_ID = ''
CLIENT_SECRET = ''
# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Coloca tus credenciales de lastfm aquí
SHARED_SECRET = ''
API_KEY = ''

last_fm_connect = requests.get(f"http://www.last.fm/api/auth/?api_key={API_KEY}")

In [5]:
def extraccion_por_paises(diccionario_zonas, años):
    todos_los_datos = [] # Esto es una lista de dicts donde se van guardando los diccionarios de cada país
    for pais in diccionario_zonas.keys():
        datos_rango = sp.search(q = f'year:{años}', limit=50, offset=0, type='track', market={pais}) # Cada una tiene que cambiar en q el rango de años
        datos_por_pais = {}
        datos_por_pais['pais'] = pais
        datos_por_pais['datos'] = datos_rango
        todos_los_datos.append(datos_por_pais)
    return todos_los_datos
   

In [6]:
def filtro_de_datos(lista_datos_zonas, datos):
    for datos_pais in lista_datos_zonas:
        for element in datos_pais['datos']['tracks']['items']:
            datos['pais'].append(datos_pais['pais'])
            datos['popularity'].append(element['popularity'])
            datos['nombre'].append(element['name'])
            artistas = []
            for artista in element['artists']:
                artistas.append(artista['name'])   
            artistas_str = ', '.join(artistas)
            datos['artista'].append(artistas_str)

In [7]:
paises_europa_central = {
    "AT": "Austria",
    "DE": "Alemania",
    "CH": "Suiza",
    "LI": "Liechtenstein",
    "BE": "Bélgica",
    "LU": "Luxemburgo",
    "NL": "Países Bajos"}

In [8]:
p_centro = extraccion_por_paises(paises_europa_central, '2000-2004')

In [9]:
dic_centro = {'nombre': [], 'popularity': [], 'artista': [], 'pais': [], 'genero': []}

In [10]:
filtro_de_datos(p_centro, dic_centro)

In [11]:
dic_centro

{'nombre': ['Unwritten',
  'Angel',
  'Mr. Brightside',
  'Rooftop',
  'Numb',
  'This Is My Time',
  'In the End',
  'Dímelo bajito',
  'Yellow',
  'La vita è',
  'Mockingbird',
  'Here She Comes Again',
  'Without Me',
  'Turn It into Something Special',
  'How You Remind Me',
  'San Pedro',
  'Faint',
  'Cabelo Raspadinho',
  'The Scientist',
  'Dar A Volta No Brasil - Ao Vivo',
  'Last Resort',
  "David's Arm - from Deuce",
  "Can't Stop",
  'Eso de saber',
  'Irgendwie, irgendwo, irgendwann',
  'Devolva-Me',
  'The Real Slim Shady',
  'He Loves U Not',
  "L'Amour Toujours",
  'Stuck in the Middle with You',
  'Somewhere Only We Know',
  '2 Faced',
  'Dancing in the Moonlight',
  'Kiedy góral umiera',
  'One Step Closer',
  'El peligro',
  'Crazy In Love (feat. JAY-Z)',
  'This Is Me',
  'Sonne',
  "American Life (Director's Cut) - 4K Remaster",
  'Denkmal',
  'Beautiful Inside',
  'Ein Kompliment',
  'Wierz? w siebie',
  'Yeah! (feat. Lil Jon & Ludacris)',
  'Take Me Home',
  'Bri

In [12]:
lista_nombres_centro = list(dic_centro.values())[0]
lista_artistas_centro = list(dic_centro.values())[2]

In [13]:
def llamada_lastfm(lista_nombres, lista_artistas, datos):
    for i in range(len(lista_nombres)):
        nombre = str(lista_nombres[i]).replace(' ', '+')
        artista = lista_artistas[i].replace(' ', '+').replace(',', '+&')
        llamada_genero = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist={artista}&track={nombre}&api_key={API_KEY}&autocorrect=1&format=json")
        try:
            genero_cancion = llamada_genero.json()['toptags']['tag'][0]['name']
            datos['genero'].append(genero_cancion)
        except KeyError:
            datos['genero'].append("NULL")
            print(f"La canción {lista_nombres[i]} no está registrada en lastfm")
            
        except IndexError:
            datos['genero'].append("NULL")
            print(f"La canción {lista_nombres[i]} no tiene tags")

In [14]:
llamada_lastfm(lista_nombres_centro, lista_artistas_centro, dic_centro)

La canción Angel no está registrada en lastfm
La canción Dímelo bajito no tiene tags
La canción San Pedro no está registrada en lastfm
La canción Dar A Volta No Brasil - Ao Vivo no tiene tags
La canción David's Arm - from Deuce no está registrada en lastfm
La canción Crazy In Love (feat. JAY-Z) no está registrada en lastfm
La canción American Life (Director's Cut) - 4K Remaster no tiene tags
La canción Wierz? w siebie no tiene tags
La canción Yeah! (feat. Lil Jon & Ludacris) no está registrada en lastfm
La canción Take Me Home no está registrada en lastfm
La canción Love Is Alive - Live; 2004 Remaster no está registrada en lastfm
La canción Till I Collapse no está registrada en lastfm
La canción Ni na no no tiene tags
La canción Angel no está registrada en lastfm
La canción Yeah! (feat. Lil Jon & Ludacris) no está registrada en lastfm
La canción Love Is Alive - Live; 2004 Remaster no está registrada en lastfm
La canción Dímelo bajito no tiene tags
La canción Respiration no está registr

In [15]:
df_centro = pd.DataFrame(dic_centro)

In [16]:
df_centro.index.name="id"

In [17]:
df_centro

,nombre,popularity,artista,pais,genero
id,,,,,
0,Unwritten,81,Natasha Bedingfield,AT,pop
1,Angel,28,"Shaggy, Rayvon",AT,NULL
2,Mr. Brightside,82,The Killers,AT,rock
3,Rooftop,0,Sasha,AT,pop
4,Numb,83,Linkin Park,AT,rock
...,...,...,...,...,...
345,Mzael Kel El Banat,0,Wael Kfoury,NL,NULL
346,Ms. Jackson,78,Outkast,NL,Hip-Hop
347,Ergaa Lel Shoua,0,Elissa,NL,NULL


In [18]:
df_centro.to_csv("paises_centro_2000_2004.csv")